# Monitoring and Troubleshooting

This notebook covers monitoring, logging, alerting, and troubleshooting for the Azure Document Intelligence pipeline.

## Configuration

Configure your Azure resources below. Resources can be in **different resource groups** - specify the appropriate resource group for each service.

In [ ]:
# ============================================
# CONFIGURATION - UPDATE THESE VALUES
# ============================================

$SUBSCRIPTION_ID = "363ef5d1-0e77-4594-a530-f51af23dbf8c"

# Resource Groups (can be different for each resource)
$FUNC_RG = "rg-docprocessing-functions-dev"
$SYNAPSE_RG = "rg-sandbox-demo-east2"
$COSMOS_RG = "rg-dlz-cosmosdb-east2-sandbox"
$STORAGE_RG = "rg-dlz-aiml-stack-dev"
$KEY_VAULT_RG = "rg-dlz-aiml-stack-dev"

# Log Analytics (may be in a different RG or subscription for centralized monitoring)
$LOG_ANALYTICS_WORKSPACE = "<YOUR_LOG_ANALYTICS_WORKSPACE>"
$LOG_ANALYTICS_RG = "<YOUR_LOG_ANALYTICS_RG>"

# Resources
$FUNC_APP_NAME = "docproc-func-dev"
$SYNAPSE_WORKSPACE = "synapse-sandbox-east2-dlz"
$COSMOS_ACCOUNT = "cosmosdb-dlz-east2-sandbox"
$STORAGE_ACCOUNT = "aimldatastore"
$KEY_VAULT_NAME = "aiml-stack-keyvault-dev"

# ============================================

az account set --subscription $SUBSCRIPTION_ID

Write-Host "Configuration set" -ForegroundColor Green
Write-Host ""
Write-Host "Resource Groups:" -ForegroundColor Cyan
Write-Host "  Function App:  $FUNC_RG"
Write-Host "  Synapse:       $SYNAPSE_RG"
Write-Host "  Cosmos DB:     $COSMOS_RG"
Write-Host "  Storage:       $STORAGE_RG"
Write-Host "  Key Vault:     $KEY_VAULT_RG"
Write-Host "  Log Analytics: $LOG_ANALYTICS_RG"

## 1. Function App Logs

In [ ]:
# View live Function App logs (streams real-time)
Write-Host "Streaming Function App logs (Ctrl+C to stop)..." -ForegroundColor Cyan

az functionapp log tail `
    --name $FUNC_APP_NAME `
    --resource-group $FUNC_RG

In [ ]:
# Get Function App metrics
Write-Host "Function App metrics (last 24 hours):" -ForegroundColor Cyan

$endTime = (Get-Date).ToString("yyyy-MM-ddTHH:mm:ssZ")
$startTime = (Get-Date).AddHours(-24).ToString("yyyy-MM-ddTHH:mm:ssZ")

az monitor metrics list `
    --resource "/subscriptions/$SUBSCRIPTION_ID/resourceGroups/$FUNC_RG/providers/Microsoft.Web/sites/$FUNC_APP_NAME" `
    --metric "FunctionExecutionCount" "FunctionExecutionUnits" "Http5xx" `
    --start-time $startTime `
    --end-time $endTime `
    --interval PT1H `
    --output table

## 2. Log Analytics Queries

Use KQL queries to analyze logs from all resources.

In [ ]:
# Get Log Analytics Workspace ID
$LAW_ID = az monitor log-analytics workspace show `
    --workspace-name $LOG_ANALYTICS_WORKSPACE `
    --resource-group $LOG_ANALYTICS_RG `
    --query customerId `
    --output tsv

Write-Host "Log Analytics Workspace ID: $LAW_ID" -ForegroundColor Green

In [ ]:
# Function App errors in last 24 hours
$query = @"
FunctionAppLogs
| where TimeGenerated > ago(24h)
| where Level == "Error"
| project TimeGenerated, FunctionName, Message, ExceptionDetails
| order by TimeGenerated desc
| take 50
"@

Write-Host "Function App Errors (last 24 hours):" -ForegroundColor Cyan

az monitor log-analytics query `
    --workspace $LAW_ID `
    --analytics-query $query `
    --output table

In [ ]:
# Cosmos DB request latency
$query = @"
AzureDiagnostics
| where ResourceProvider == "MICROSOFT.DOCUMENTDB"
| where Category == "DataPlaneRequests"
| where TimeGenerated > ago(24h)
| summarize 
    AvgLatency = avg(duration_s),
    P95Latency = percentile(duration_s, 95),
    RequestCount = count()
    by bin(TimeGenerated, 1h)
| order by TimeGenerated desc
"@

Write-Host "Cosmos DB Latency (last 24 hours):" -ForegroundColor Cyan

az monitor log-analytics query `
    --workspace $LAW_ID `
    --analytics-query $query `
    --output table

In [ ]:
# Synapse pipeline failures
$query = @"
SynapseIntegrationPipelineRuns
| where TimeGenerated > ago(7d)
| where Status == "Failed"
| project TimeGenerated, PipelineName, RunId, FailureType, ErrorMessage
| order by TimeGenerated desc
"@

Write-Host "Synapse Pipeline Failures (last 7 days):" -ForegroundColor Cyan

az monitor log-analytics query `
    --workspace $LAW_ID `
    --analytics-query $query `
    --output table

In [ ]:
# Key Vault access audit
$query = @"
AzureDiagnostics
| where ResourceProvider == "MICROSOFT.KEYVAULT"
| where Category == "AuditEvent"
| where TimeGenerated > ago(24h)
| project TimeGenerated, OperationName, ResultType, CallerIPAddress, identity_claim_upn_s
| order by TimeGenerated desc
| take 50
"@

Write-Host "Key Vault Access Audit (last 24 hours):" -ForegroundColor Cyan

az monitor log-analytics query `
    --workspace $LAW_ID `
    --analytics-query $query `
    --output table

## 3. Common Issues and Solutions

### Subscription Access Errors

In [ ]:
# Check Azure login status
Write-Host "Current Azure account:" -ForegroundColor Cyan
az account show --query "{Name:name, ID:id, User:user.name}" --output table

# If not logged in:
# az login

In [ ]:
# List available subscriptions
Write-Host "Available subscriptions:" -ForegroundColor Cyan
az account list --query "[].{Name:name, ID:id, State:state}" --output table

### Function Deployment Errors

In [ ]:
# Check Function App status
Write-Host "Function App status:" -ForegroundColor Cyan

az functionapp show `
    --name $FUNC_APP_NAME `
    --resource-group $FUNC_RG `
    --query "{Name:name, State:state, Runtime:siteConfig.linuxFxVersion, HostNames:hostNames}" `
    --output table

In [ ]:
# Restart Function App
Write-Host "Restarting Function App..." -ForegroundColor Cyan

az functionapp restart `
    --name $FUNC_APP_NAME `
    --resource-group $FUNC_RG

Write-Host "Function App restarted" -ForegroundColor Green

### Rate Limit Errors (429)

In [ ]:
# Check Document Intelligence rate limits
Write-Host "Document Intelligence has a default rate limit of 15 TPS" -ForegroundColor Yellow
Write-Host "`nTo reduce rate limit errors:" -ForegroundColor Cyan
Write-Host "  1. Reduce MAX_CONCURRENT_REQUESTS in Function App settings"
Write-Host "  2. The function implements exponential backoff automatically"
Write-Host "  3. Contact Azure support to increase quota"

In [ ]:
# Check current MAX_CONCURRENT_REQUESTS setting
Write-Host "Current Function App settings:" -ForegroundColor Cyan

az functionapp config appsettings list `
    --name $FUNC_APP_NAME `
    --resource-group $FUNC_RG `
    --query "[?name=='MAX_CONCURRENT_REQUESTS'].{Name:name, Value:value}" `
    --output table

In [ ]:
# Reduce concurrent requests if needed
Write-Host "Updating MAX_CONCURRENT_REQUESTS to 5..." -ForegroundColor Cyan

# Uncomment to apply:
# az functionapp config appsettings set `
#     --name $FUNC_APP_NAME `
#     --resource-group $FUNC_RG `
#     --settings MAX_CONCURRENT_REQUESTS=5

Write-Host "Uncomment the command above to apply" -ForegroundColor Yellow

### Cosmos DB Partition Key Errors

In [ ]:
# Check Cosmos DB container partition key
Write-Host "Cosmos DB container configuration:" -ForegroundColor Cyan

az cosmosdb sql container show `
    --account-name $COSMOS_ACCOUNT `
    --database-name "DocumentsDB" `
    --name "ExtractedDocuments" `
    --resource-group $COSMOS_RG `
    --query "resource.{Id:id, PartitionKey:partitionKey.paths[0], IndexingMode:indexingPolicy.indexingMode}" `
    --output table

### Function Timeout Issues

In [ ]:
Write-Host "Function Timeout Information:" -ForegroundColor Cyan
Write-Host @"

Large PDFs can take 30+ seconds to process.

Timeout settings:
  - Default: 230 seconds
  - Maximum (Consumption): 10 minutes
  - Maximum (Premium/Dedicated): Unlimited

To increase timeout, update host.json:
  {
    "functionTimeout": "00:10:00"
  }

"@

## 4. Create Alerts

In [ ]:
# Create alert for Function App errors
Write-Host "Creating alert for Function App errors..." -ForegroundColor Cyan

$alertQuery = @"
FunctionAppLogs
| where Level == 'Error'
| summarize ErrorCount = count() by bin(TimeGenerated, 5m)
| where ErrorCount > 5
"@

Write-Host "Alert query:" -ForegroundColor Yellow
Write-Host $alertQuery

Write-Host "`nTo create this alert, run:" -ForegroundColor Cyan
Write-Host @"
az monitor scheduled-query create `
  --resource-group $RESOURCE_GROUP `
  --name "FunctionAppErrors" `
  --scopes "<LOG_ANALYTICS_WORKSPACE_RESOURCE_ID>" `
  --condition "count > 5" `
  --condition-query "<QUERY>" `
  --evaluation-frequency 5m `
  --window-size 15m `
  --severity 2
"@

## 5. Health Check Dashboard

In [ ]:
Write-Host "========================================" -ForegroundColor Cyan
Write-Host "   Pipeline Health Check" -ForegroundColor Cyan  
Write-Host "========================================" -ForegroundColor Cyan

# Check Function App
Write-Host "`n[Function App]" -ForegroundColor Yellow
Write-Host "  Resource Group: $FUNC_RG" -ForegroundColor Gray
$funcStatus = az functionapp show --name $FUNC_APP_NAME --resource-group $FUNC_RG --query state -o tsv 2>$null
if ($funcStatus -eq "Running") {
    Write-Host "  Status: Running" -ForegroundColor Green
} else {
    Write-Host "  Status: $funcStatus" -ForegroundColor Red
}

# Check health endpoint
try {
    $health = Invoke-RestMethod -Uri "https://$FUNC_APP_NAME.azurewebsites.net/api/health" -Method Get -TimeoutSec 10
    Write-Host "  Health endpoint: OK" -ForegroundColor Green
} catch {
    Write-Host "  Health endpoint: FAILED" -ForegroundColor Red
}

# Check Cosmos DB
Write-Host "`n[Cosmos DB]" -ForegroundColor Yellow
Write-Host "  Resource Group: $COSMOS_RG" -ForegroundColor Gray
$cosmosStatus = az cosmosdb show --name $COSMOS_ACCOUNT --resource-group $COSMOS_RG --query "provisioningState" -o tsv 2>$null
if ($cosmosStatus -eq "Succeeded") {
    Write-Host "  Status: Healthy" -ForegroundColor Green
} else {
    Write-Host "  Status: $cosmosStatus" -ForegroundColor Red
}

# Check Synapse
Write-Host "`n[Synapse Workspace]" -ForegroundColor Yellow
Write-Host "  Resource Group: $SYNAPSE_RG" -ForegroundColor Gray
$synapseStatus = az synapse workspace show --name $SYNAPSE_WORKSPACE --resource-group $SYNAPSE_RG --query "provisioningState" -o tsv 2>$null
if ($synapseStatus -eq "Succeeded") {
    Write-Host "  Status: Healthy" -ForegroundColor Green
} else {
    Write-Host "  Status: $synapseStatus" -ForegroundColor Red
}

# Check Storage Account
Write-Host "`n[Storage Account]" -ForegroundColor Yellow
Write-Host "  Resource Group: $STORAGE_RG" -ForegroundColor Gray
$storageStatus = az storage account show --name $STORAGE_ACCOUNT --resource-group $STORAGE_RG --query "provisioningState" -o tsv 2>$null
if ($storageStatus -eq "Succeeded") {
    Write-Host "  Status: Healthy" -ForegroundColor Green
} else {
    Write-Host "  Status: $storageStatus" -ForegroundColor Red
}

# Check Key Vault
Write-Host "`n[Key Vault]" -ForegroundColor Yellow
Write-Host "  Resource Group: $KEY_VAULT_RG" -ForegroundColor Gray
$kvStatus = az keyvault show --name $KEY_VAULT_NAME --resource-group $KEY_VAULT_RG --query "properties.provisioningState" -o tsv 2>$null
if ($kvStatus -eq "Succeeded") {
    Write-Host "  Status: Healthy" -ForegroundColor Green
} else {
    Write-Host "  Status: $kvStatus" -ForegroundColor Red
}

Write-Host "`n========================================" -ForegroundColor Cyan

## Troubleshooting Reference

| Error | Cause | Solution |
|-------|-------|----------|
| `Unauthorized` (401) | Function key invalid | Store key in Key Vault, grant Synapse access |
| `Forbidden` (403) | Missing permissions | Check RBAC and Key Vault policies |
| `Could not download file` | Missing SAS token | Function auto-generates; check storage connection |
| `Rate limit` (429) | Too many requests | Reduce MAX_CONCURRENT_REQUESTS |
| `Timeout` | Large PDF processing | Increase function timeout in host.json |
| `Partition key error` | Missing sourceFile | Ensure documents have sourceFile field |
| `Can't determine language` | Missing --python flag | Use `func publish --python` |